In [ ]:
# Required Python Packages
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from pandas import ExcelWriter
from sklearn.metrics import accuracy_score,precision_score, recall_score, average_precision_score, precision_recall_curve


In [ ]:
train_dataset = pd.read_excel('sample_input.xlsx',sheet_name='Simplified v1 Data')

le = preprocessing.LabelEncoder()
tvec = TfidfVectorizer(min_df=1, stop_words='english', max_features=2000)
X = tvec.fit_transform(train_dataset['Feedback Details'])
train_dataset['Category'] = train_dataset['Category'].str.lower()
print(len(train_dataset.index))
#Get records which are having blank null values for Sentiment column.
blank_values_feedback_cat = train_dataset[train_dataset['Category'].isnull()]
#Get records which are having non null values for Sentiment column.
train_dataset = train_dataset[train_dataset['Category'].notnull()]
print(len(train_dataset.index))
le.fit(train_dataset['Category'])
ps = SnowballStemmer('english')


In [ ]:
def perform_stem(x):
    sent=""
    sentence_list=""
    words = word_tokenize(x)
    for w in words:
        sent = sent+" "+ps.stem(w)
    sentence_list = sentence_list+sent
    return sentence_list

In [ ]:
train_dataset['Feedback Details'].apply(lambda x : perform_stem(x))

In [ ]:
test_dataset = pd.read_excel('test_dataset.xlsx',sheet_name='test_dataset')

In [ ]:
test_dataset['Category'] = test_dataset['Category'].str.lower()

In [ ]:
train_x=tvec.transform(train_dataset['Feedback Details'])
test_x=tvec.transform(test_dataset['Feedback Details'])

In [ ]:
train_y=le.transform(train_dataset['Category'])

In [ ]:
model = RandomForestClassifier(n_estimators=500,n_jobs=-1)
trained_model = model.fit(train_x, train_y)
predictions = trained_model.predict(test_x)

In [ ]:
predictions

In [ ]:
test_dataset.reset_index(inplace=True,drop=True)

In [ ]:
inv_predictions=le.inverse_transform(predictions)
pp=pd.Series(inv_predictions)

In [ ]:
test_dataset['Predicted_Category']=pp

In [ ]:
test_dataset

In [ ]:
exel_writer=ExcelWriter('output/test_dataset_category.xlsx')
ex = test_dataset.to_excel(excel_writer=exel_writer, index=False)
exel_writer.save()


In [ ]:
arr =  le.classes_
lst = arr.tolist()
list2 = ['PS_'+x for x in lst]
print(list2)

In [ ]:
def compare(x):
    if x['Predicted_Category'] == x['Category']:
        return True
    elif x['Category'] == 'null':
        return 'Not Applicable'
    else:
        return False

In [ ]:
test_dataset['Match'] = test_dataset.apply(compare, axis=1)

In [ ]:
probs = pd.DataFrame(model.predict_proba(test_x),columns=list2)
dd = pd.concat([test_dataset,probs], axis=1)
# print ("all probabilities:\n", probs, "\n")

In [ ]:
dd.to_excel("test_dataset_confidence_score_category.xlsx", index=False)